In [1]:
import joblib
import pandas as pd
import numpy as np
import sparse
import json

### Meds

In [2]:
df_time_series = joblib.load('sample_output/out_meds/sdf.joblib')
discretization_bins = json.load(open('metadata/out_meds/discretization.json', 'r'))
feature_names = json.load(open('metadata/out_meds/X_all.feature_names.json', 'r'))
len(discretization_bins)

1986

In [3]:
cols_useful = list(discretization_bins.keys())

In [4]:
df_time_series = df_time_series[cols_useful]

In [5]:
df_time_series

variable_name,154252_route_value,154253_route_value,154252_dose_value,154254_route_value,154253_dose_value,154254_dose_value,154255_route_value,154255_dose_value,154256_route_value,154256_dose_value,154257_route_value,154257_dose_value,154259_route_value,154259_dose_value,154265_dose_value,154265_route_value,154271_route_value,154271_dose_value,154272_route_value,154272_dose_value,154273_route_value,154273_dose_value,154274_route_value,154274_dose_value,154278_route_value,154278_dose_value,154279_dose_value,154279_route_value,154280_route_value,154280_dose_value,154281_route_value,154281_dose_value,154285_route_value,154285_dose_value,154288_route_value,154288_dose_value,154291_route_value,154291_dose_value,154293_route_value,154293_dose_value,154297_route_value,154297_dose_value,154299_route_value,154299_dose_value,154300_route_value,154300_dose_value,154304_route_value,154304_dose_value,154306_dose_value,154306_route_value,...,519089_dose_value,519089_route_value,520139_route_value,520139_dose_value,525531_route_value,525531_dose_value,525995_route_value,526935_dose_value,525995_dose_value,526935_route_value,531466_dose_value,531466_route_value,531498_dose_value,531498_route_value,652664_dose_value,652664_route_value,654324_dose_value,654324_route_value,655148_dose_value,655148_route_value,662425_dose_value,662425_route_value,663869_route_value,663869_dose_value,664519_dose_value,664519_route_value,666536_dose_value,666536_route_value,667734_dose_value,667734_route_value,670121_dose_value,670121_route_value,680185_route_value,680185_dose_value,680198_route_value,680198_dose_value,681989_dose_value,681989_route_value,681994_route_value,681994_dose_value,683395_dose_value,683395_route_value,686603_dose_value,686603_route_value,692199_route_value,692199_dose_value,694267_dose_value,694267_route_value,694268_route_value,694268_dose_value
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [6]:
ddf = df_time_series.sparse.to_dense()

In [7]:
from steps_2 import *
from helpers import *

In [8]:
df = ddf

In [9]:
pool = multiprocessing.Pool(multiprocessing.cpu_count())
out_0 = list(tqdm(pool.imap_unordered(
    smart_qcut_bins,
    [(df[col], discretization_bins[col]) for col in df.columns]), total=len(df.columns)
))
cols_data, dis_bins = zip(*out_0)
out = list(tqdm(pool.imap_unordered(dummify, cols_data), total=len(df.columns)
))
pool.close()
pool.join()

100%|██████████| 1986/1986 [00:01<00:00, 1616.37it/s]


In [10]:
df_features = pd.concat(out, axis=1).sort_index(axis=1)

In [11]:
# Drop any values='missing' for now
df_features = df_features.loc[:, [col for col in df_features.columns if 'missing' not in col]]

In [12]:
# Make sure same features
set(df_features.columns) <= set(feature_names)

True

In [13]:
df_features = df_features.reindex(columns=feature_names, fill_value=0)

In [14]:
df_features

,"154252_dose_value_(0.039, 0.4]","154252_dose_value_(0.4, 10.0]",154252_route_value_SL,"154253_dose_value_(30.0, 60.0]","154253_dose_value_(9.999, 30.0]",154253_route_value_IM,154253_route_value_IV,"154254_dose_value_(325.0, 975.0]","154254_dose_value_(80.999, 325.0]",154254_route_value_ORAL,154254_route_value_PER G TUBE,154254_route_value_PER NG TUBE,154254_route_value_PER OG TUBE,"154255_dose_value_(0.499, 5.0]","154255_dose_value_(12.5, 21.0]","154255_dose_value_(5.0, 7.5]","154255_dose_value_(7.5, 12.5]",154255_route_value_IV,"154256_dose_value_(199.999, 400.0]","154256_dose_value_(400.0, 600.0]","154256_dose_value_(600.0, 800.0]",154256_route_value_ORAL,154256_route_value_PER G TUBE,154256_route_value_PER J TUBE,154256_route_value_PER NG TUBE,154256_route_value_PER OG TUBE,"154257_dose_value_(449.999, 750.0]",154257_route_value_IV,"154259_dose_value_(19.999, 40.0]","154259_dose_value_(40.0, 80.0]",154259_route_value_ORAL,154259_route_value_PER G TUBE,154259_route_value_PER NG TUBE,154259_route_value_PER OG TUBE,154265_dose_value_10.0,154265_dose_value_20.0,154265_route_value_IV,"154271_dose_value_(15.0, 30.0]","154271_dose_value_(7.499, 15.0]",154271_route_value_IM,154271_route_value_IV,"154272_dose_value_(14.999, 60.0]",154272_route_value_ORAL,"154273_dose_value_(10.0, 180.0]","154273_dose_value_(4.999, 10.0]",154273_route_value_ORAL,154273_route_value_SWISH & SPIT,154273_route_value_SWISH & SWAL,"154274_dose_value_(17.0, 255.0]","154274_dose_value_(4.079, 17.0]",...,664519_route_value_PER NG TUBE,"666536_dose_value_(0.499, 2.0]","666536_dose_value_(2.0, 4.0]",666536_route_value_IV,"667734_dose_value_(0.099, 1.2]","667734_dose_value_(1.2, 2.1]","667734_dose_value_(2.1, 3.1]","667734_dose_value_(3.1, 4.7]","667734_dose_value_(4.7, 23.9]",667734_route_value_IV,670121_dose_value_1.0,670121_dose_value_3.0,670121_route_value_IART,670121_route_value_IV,"680185_dose_value_(1.999, 3.0]","680185_dose_value_(3.0, 4.0]","680185_dose_value_(4.0, 3000.0]",680185_route_value_IV,"680198_dose_value_(0.999, 4.0]",680198_route_value_ORAL,681989_dose_value_4.0,681989_dose_value_9.0,681989_route_value_IV,"681994_dose_value_(0.399, 2.5]","681994_dose_value_(12.5, 15.0]","681994_dose_value_(2.5, 5.0]","681994_dose_value_(5.0, 7.5]","681994_dose_value_(7.5, 12.5]",681994_route_value_IV,"683395_dose_value_(2.999, 20.0]","683395_dose_value_(20.0, 30.0]","683395_dose_value_(30.0, 50.0]","683395_dose_value_(50.0, 80.0]","683395_dose_value_(80.0, 300.0]",683395_route_value_IV,686603_dose_value_0.5,686603_route_value_IM,"692199_dose_value_(0.999, 2.0]","692199_dose_value_(2.0, 2000.0]",692199_route_value_IV,694267_dose_value_1.0,694267_dose_value_2.0,694267_route_value_FTUBE,694267_route_value_ORAL,"694268_dose_value_(0.019, 1.5]","694268_dose_value_(1.5, 2.5]","694268_dose_value_(2.5, 3.5]","694268_dose_value_(3.5, 6.0]","694268_dose_value_(6.0, 100.0]",694268_route_value_IV
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [15]:
sdf = df_features.astype(pd.SparseDtype(int, fill_value=0))
X_all = sparse.COO(sdf.sparse.to_coo())

In [17]:
sparse.save_npz('sample_output/out_meds/X_all.npz', X_all)
df_time_series[[]].to_csv('sample_output/out_meds/X_all.IDs.csv')